<img style="float: left" width="20%" src="../images/erasmus.png">
<img style="float: right" width="20%" src="../images/surfsara.png">
<hr style="clear: both" />

## Week 5 - Introduction to RDDs - Apache Spark

This notebook provides an introduction into Apache Spark RDD API using PySPark. Press Shift-Enter to execute the code. You can use code completion by using tab.

During the exercises you may want to refer to [The PySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html) for more information on possible transformations and actions. We will provide links to the documentation when we introduce methods on RDDs.

## Handing in and deadline
  
You can download this notebook by clicking File -> Download as -> Notebook(.ipynb). Then upload the file in Blackboard under assignments for the fifth week.

The deadline for this assignment is **January 5 2018**

## 1. The SparkContext

The SparkContext contains all the information about the way Spark is set up. When running on a cluster, the SparkContext contains the address of the cluster and will make sure operations on RDDs will be executed there. In the cell below, we create a [`SparkContext`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext) using `local mode`. This means that Spark will run locally, not on a cluster. It will offer some form of parallelism by making use of the various cores it has available.

Note, that Spark is best used in `cluster mode` where it will run on many machines simultaneously. `Local mode` is only meant for training or testing purposes. However, Spark works quite well in local mode and can be quite powerful. In order to run locally developed code on a cluster, the only thing that needs to be changed is the `SparkContext` and paths to in- and output files.

Even when working in `local mode` it is important to think of an RDD as a data structure that is distributed over many machines on a cluster, and is not available locally. The machine that contains the `SparkContext` is called the *driver*. The SparkContext will communicate with the cluster manager to make sure that the operations on RDDs will run on the cluster. It is important to realize that the driver is a separate entity from the nodes in the cluster. You can consider the notebook as being the driver.

In [ ]:
# initialize Spark
from pyspark import SparkContext, SparkConf
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

if not 'sc' in globals(): # This 'trick' makes sure the SparkContext sc is initialized exactly once
    conf = SparkConf().setMaster('local[*]')
    sc = SparkContext(conf=conf)

## Creating an RDD

There are three ways to create an RDD. By transforming an existing one, by reading in data, or by creating an RDD based on a local data structure. We show this last option below.

A Python list containing some words is used to create an RDD by calling [`parallelize`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.parallelize), a method of `SparkContext`. This list is very small and will not benefit from the parallelism of Spark. 

We then print the number of records in the RDD, by calling the `count()` method.

In [ ]:
wordsList = ['Dog', 'Cat', 'Rabbit', 'Hare', 'Deer', 'Gull', 'Woodpecker', 'Mole']
wordsRDD = sc.parallelize(wordsList)
print(wordsRDD.count())

## 2. Map transformation 

There are two kinds of operations on RDDs: transformations and actions. Transformations take as input an RDD and produce as output another RDD. (You cannot change an existing RDD, they are immutable.) Computation of transformations is deferred until an *action* needs to be executed. An action does not return an RDD but returns data to the driver, or writes data to disk or a database.

This *laziness* of executing transformations allows Spark to optimize computations. Only when the user wants real output, the framework will start to compute.

One of the most used transformations is [`Map`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map). This is very similar to the `Map` in MapReduce. It will transform each element in the RDD according to the function that is given as an input. The function that is input to `map` should have a single input argument.

Below we want to change all words in the wordsRDD to their plural form. We will do this using a [map](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=map#pyspark.RDD.map) transformation.
Remember that the function itself is only defined for a single word. `Map` will apply the function to each element of the RDD. 

First, we present a simple Python function that takes a single word as argument and returns the word with an 's' added to it. In the next step we will use this function in a map transformation of the wordsRDD.

Take a look at the function definition below and execute it.

In [ ]:
def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's' to word.  

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return word + 's'

# Let's see if it works

print(makePlural('cat'))

Next, we want to use the `makePlural` function as input for the `map` transformation on wordsRDD.
The action [collect()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=collect#pyspark.RDD.collect) transfers the content of the RDD to the driver. The result of `collect()` is  available to our local environment in Python. It is not an RDD but a Python list!

Note, that a large RDD may be scattered over many machines. In such a case calling `collect()` may not be a good idea (you do pay the charge anyway). 

## Exercise 1

In the cell below enter the name of the function that map should apply to each element of the RDD in order to end up with an RDD of words in plural form.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
pluralRDD = wordsRDD.map(<FILL IN>)

#see if it works
print(pluralRDD.collect())

## Using lambda functions

We can achieve the same functionality by using lambda functions. In this case we define `makePlural` not using `def` as we did above, but as an anonymous function that we define inside `map`. 

## Exercise 2

Try this in the next cell.

In [ ]:
# Replace <FILL IN> with a lambda function that adds s at the end of a string

lambdaPluralRDD = wordsRDD.map(<FILL IN>)
output = lambdaPluralRDD.collect()

#see what the output is
print(output)

## Exercise 3

Another transformation is [filter()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=filter#pyspark.RDD.filter). It takes as argument a predicate function (a function that is evaluated to true or false) and does what is says it does; filter. It applies the predicate to all elements of the RDD and lets only those pass that return true.

Use the [filter()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=filter#pyspark.RDD.filter) method of RDD to keep only words with a length larger than three. Use a lambda function to write a predicate that does this.

Next, [Count()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=count#pyspark.RDD.count) the number of words. 

[Count()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=count#pyspark.RDD.count) has been used above and is an example of an action. Remember that actions trigger Sparks computations. Transformations are evaluated lazily and their computation is deferred until an action is called.

There should be 6 words that pass the filter. 

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

filteredRDD = wordsRDD.filter(<FILL IN>)
<FILL IN>

## Exercise 4

Let's do another map transformation on wordsRDD. For each word in wordsRDD determine its length, again using a lambda function. 

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
wordLengths = <FILL IN>.collect()
print(wordLengths)

## Pair RDDs

Pair RDDs are very important within the Spark RDD API. Each element of a Pair RDD is a pair `(x,y)` where `x` is interpreted as being the key and `y` as the value. Spark offers quite a number of `...byKey` and `...byValues` methods that operate on pairRDDs. As we will see, these methods can be used to define functions per key, very similar to Hadoops MapReduce.

*(Remember that both the key and value can be of any type; they can be tuples, lists or pairs etc.)* 

Below we define a Python string variable called `sonnet`. It is assigned Shakespeare's first sonnet in the form of a single line of text. The character `\` is used to let Python ignore the new line character. 

Execute the cell, otherwise the variable is not declared and assigned a value.

In [ ]:
sonnet = "From fairest creatures we desire increase, \
That thereby beauty\'s rose might never die, \
But as the riper should by time decease, \
His tender heir might bear his memory: \
But thou contracted to thine own bright eyes, \
Feed'st thy light's flame with self-substantial fuel, \
Making a famine where abundance lies, \
Thy self thy foe, to thy sweet self too cruel: \
Thou that art now the world's fresh ornament, \
And only herald to the gaudy spring, \
Within thine own bud buriest thy content, \
And, tender churl, mak'st waste in niggarding: \
Pity the world, or else this glutton be, \
To eat the world\'s due, by the grave and thee."

## Python magic

From this text we first remove punctuation. The next cell is just Python. You may want to skip this if your focus is just on Spark, but don't forget to execute the cell.

`maketrans()` is a Python method on strings that very efficiently can make character substitutions. Below we use it to remove all punctuation characters. The curly braces indicate a dictionary, and the expression within it, is called a comprehension. The result is a dictionary of key-value pairs, called table, where the key is a punctuation character and the value is `None`. When making substitutions by means of `translate` this table then removes all the entries that have a `None` value.

In [ ]:
import string

table = str.maketrans({key: None for key in string.punctuation})

s = "string. With. Punctuation?"
new_s = s.translate(table)

new_s

## Parallelizing the text

In the next cell a lot is happening in one line. The text above is first translated - which in this case means that each punctuation character is removed. Then on the result, the `lower()` method is applied. (This is a Python method on strings.) This puts a string in lowercase letters. Then this result is `split()`, meaning that the text is split in individual words. (Also a Python method on strings.) This results in a list of words, all lowercase, with no punctuation. This is input to the `parallelize()` method which turns it into an RDD.

*Calling consecutive methods by using dot-notation is called chaining. It is possible of course to execute these steps individually, but chaining can be very convenient, especially in Spark. Consider the individual steps: first parallelize the text, then map the resulting RDD to remove the punctuation, then map the resulting RDD to lowercase the text and then map the resulting RDD of that step to split the data... Doing this instead by chaing methods safes a lot of typing.* 

To show just the 5 first elements, we use [`take()`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take). This limits the amount of data that is sent to the driver.

In [ ]:
textRDD = sc.parallelize(sonnet.translate(table).lower().split())

textRDD.take(5)

## Exercise 5

What would happen if we wouldn't split the text but directly transform it into an RDD? Try this in the next cell (omit `translate` and `lower` as well).

Try to predict what will happen. Remember that a string in Python is very similar to a list. 

(For a list called `mylist` the first element is given by `mylist[0]`. Similarly `mystring[0]` will return the first character of the string `mystring`.)

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

anotherRDD = sc.<FILL IN>
anotherRDD.collect()

## Exercise 6

Yes we are going to count the words in textRDD. As a first step do the following.
Transform every word in textRDD into a pair (word, 1). Use a lambda function.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

pairRDD = textRDD.<FILL IN>
pairRDD.take(5)

## Exercise 7

There is an *action* called [countByKey](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.countByKey) that performs the counting and returns it as a Python dictionary.
Use it below to see the counts.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

dict = <FILL IN>

## Below is python code that nicely prints the dictionary

wlist =sorted(dict.items(), key=lambda x: -x[1])
for i in range(10):
    print(wlist[i])

##  reduceByKey

The action `countByKey` is nice but it is an action, meaning that we do not have the wordcounts available as an RDD. When we have a large number of counts, and we use `countByKey` all of these are sent back to the driver in the form of a Python dict, which might not fit in local memory.

If we want to count words and keep the result into an RDD we have to use the [reduceByKey](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=reducebykey#pyspark.RDD.reduceByKey) *transformation*.

This transformation works almost exactly like Reduce in Hadoops MapReduce. It expects the RDD to consist of key value pairs an it will perform a reduce operation *per key*. 

As input [reduceByKey](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=reducebykey#pyspark.RDD.reduceByKey) takes a *two-argument function* that will be applied on the values when they are grouped by key. 

Remember that a reduce function needs two arguments and will reduce all elements of a list to a single value.  

## Exercise 8

Next, create a lambda function that does the counting and forms the input for `reduceByKey`.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
# Note that reduceByKey takes in a function that accepts two values and returns a single value
# The function that is input to reduceByKey only works on the values. Spark will execute this function per key

wordCounts = pairRDD.reduceByKey(<FILL IN>)
print(wordCounts.collect())

Instead of using `collect` we can use [takeOrdered](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeOrdered) to see the most frequent words first.

Below we show 10 elements from the RDD. The elements are pairs and we sort them by the second element (denoted by `x[1]` in the lambda function. The minus indicates descending order.

In [ ]:
wordCounts.takeOrdered(10, lambda x: -x[1])

## 3. Analysing tweets

We saw how to analyse tweets in MongoDB, or at least how to query them. Spark is even more powerful and certainly works much better on a large scale.

We also show now how to read in a file and transform it into an RDD. We read in a file of Dutch tweets by making use of [sc.textFile](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=textfile#pyspark.SparkContext.textFile). This will interpret the contents of the file as text, not as JSON. We later have to explicitly tell Python that the text should be interpreted as JSON. We'll do this in exercise 10.

(Spark has support for JSON files when working with Data Frames, as we will see later.)

In the file, every tweet is on a single line. `textFile` will create an RDD, where each line of the input file will be an element of the RDD. The file is a local file in our case. Often when using Spark files reside on a distributed file system like HDFS. When creating the RDD Spark may distribute the data over many machines.

First, let's look at the first line of the data file we are going to use. We use a simple Unix command here (no Python) to view the first line of a file that resides on local disk. Notice, that this is a single tweet in JSON. 

In [ ]:
# Unix bash command called head.
# The ! announces a Unix command is coming to Jupyter

!head -1 ../data/tweets.json

## Exercise 9

Below the call to `textFile` is made. There is also a call to `filter` to make sure empty lines are not included.

Print out the first tweet in the RDD by making use of the [take](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=take#pyspark.RDD.take) action. Very easy.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
# Use take to print out the first tweet

tweetRDD = (sc.textFile('../data/tweets.json')
            .filter(lambda x : len(x) > 0))
     
print(tweetRDD.<FILL IN>)

## Conversion to json

Next, we are going to convert the tweets into JSON format. For this purpose we import the json Python library. In Python a string `s` is converted into json by calling `json.loads(s)`.

Converting the tweets into JSON will return a Python dictionary where each key is an attribute of the tweet. Some attributes, like *user* have sub-attributes.

In the next cell the conversion will take place and the first tweet is shown. 

## Exercise 10

Transform every tweet in the tweetRDD to json.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

import json

jsonTweetRDD = tweetRDD.<FILL IN>

# Print out the first element (tweet) of the jsonTweetRDD (nicely indented)

parsed = jsonTweetRDD.take(1)
print(json.dumps(parsed, indent=2))

## Access to fields in the tweets

In the cell below some fields from the tweets are selected. Notice that the input `x` for the lambda function is a tweet in json format (a Python dictionary). The result of the lambda function (defined after `:`) is a list with values of the selected fields from the tweet.

You should be able to figure out how to select information from a tweet, after looking at this example.

In [ ]:
jsonTweetRDDtext = jsonTweetRDD.map(lambda x: [x['lang'],
                                               x['entities']['hashtags'],
                                               x['user']['name'], 
                                               x['user']['screen_name'], 
                                               x['user']['followers_count'], 
                                               x['user']['description']])
jsonTweetRDDtext.take(1)

## Selecting Text

We will work with the text of the tweets in the next few cells.

## Exercise 11

From the jsonTweetRDD select only the text of the tweets. (Do not put the text in a list, like we did above with the  fields we selected there.)

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

tweetTextRDD = (jsonTweetRDD.<FILL IN>
                
tweetTextRDD.take(1)

## TweetTokenizer

The advantage of the RDD API is that it works well with unstructured data like text. We can use any function to transform RDDs. For example, to split text into words or tokens (tokenization) is often more difficult than just calling `split()` on a string. Especially for text in Tweets determining what tokens are is completely different from doing the same with for example newspaper text.

We can make use of the Python [NLTK](http://www.nltk.org/api/nltk.tokenize.html) library for tokenization of tweets. The NLTK (Natural Language Tool Kit) contains a `TweetTokenizer` that is specially build to tokenize tweets.

In the cell below we show a short example in Python code (no RDDs!). A `TweetTokenizer` is created and used to tokenize an example tweet text in the variable s. The result is a list of tokens. Try it.

In [ ]:
from nltk.tokenize import TweetTokenizer

tweetTknzr = TweetTokenizer()
s = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"
tweetTknzr.tokenize(s)

## Exercise 12

Use `tweetTknzr` to tokenize all tweets in the RDD. Print the first 10 tokens, using `take`.

Note: we want `take` to return the first 10 tokens, not the first 10 *lists* of tokens. When you get lists you have to flatten them...

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
# two lines should do it...

<FILL IN>
<FILL IN>

## Exercise 13

Count the tokens in the tweets, but only those with a length larger than 2. Show the top 15 in descending frequency.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
# use as many lines as you like


## 4. PageRank in wikipedia

Below we present a (naive) implementation of PageRank in Spark. The exact working of the algorithm has been explained before. Here we lay out the basic steps in terms of RDDs.

We start with a file that contains pairwise urls, obtained from [wikispeedia](http://snap.stanford.edu/data/wikispeedia.html), a game were users were asked to navigate from a given source to a given target article, by only clicking Wikipedia links.

This file is parsed with a helper function `parseNeighbours`. The resulting RDD (not shown in the picture) is a pair RDD, called **lines** in the code. The entries here are the same as in the file: URL pairs.
After calling `distinct` to remove double entries, the entries are grouped by key. The result is the **RDD links** (left in the picture). The initial **RDD ranks** is created by a simple map.

<img style="float: center" width="80%" src="https://github.com/abbas-taher/pagerank-example-spark2.0-deep-dive/raw/master/images/img-3.jpg"> 

In [ ]:
# Two functions for pageRank

from operator import add
import re

# Distribute the PageRank score over the given urls

def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls: yield (url, rank / num_urls)

# Parse the input file as url-url pairs

def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

First, let us see what is in the file that contains the links. (We use a Unix command here.)

You will see a tab delimited file with two entries per line. Both entries are Wikipedia Articles, although the left side is a bit hard to read. 

A pair indicates that a user at wikispeedia went from the article at the left to the article at the right.

In [ ]:
# Let's do some Unix : the ! character announces a Unix command to Jupyter
!head ../data/links.tsv

### Reading the links into Spark

We make use of sc.textFile to read in the data. Every line will be an element of this RDD.

In [ ]:
lines = sc.textFile('../data/links.tsv')

## Exercise 14

Show the first 10 elements of the lines RDD. 

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

<FILL IN>

In order to get a list of links for each article (like ** RDD links** in the picture) we have to parse the file, remove duplicate entries and group by the first entry, the key. 

This happens is the next cell. Notice the chaining of the commands. `parseNeighbors` is one of the helper functions dedined above. What [`distinct()`](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=groupbykey#pyspark.RDD.distinct) and [`groupByKey()`](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=groupbykey#pyspark.RDD.groupByKey) do should be quite obvious. For details click the links.

The result is the **RDD links** in the picture. It plays a role in the iteration and for that reason we persist the RDD in memory by calling [`cache()`](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=groupbykey#pyspark.RDD.cache). 

In [ ]:
# Loads all URLs from input file and initialize their neighbors.
links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()

## Exercise 15

Show the first 10 entries of the links RDD. You can use `take` but it will only get you half way there...

In order to do this properly you have to convert the values of this RDD into a Python list. There is a method called [mapValues](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=mapvalues#pyspark.RDD.mapValues) that acts as a map on the value of a key-value pair. The (Python) function to convert another object to a list is called `list`. (`list(object)` will turn return a list based on the value of `object`.) 

You may want to do this in two steps. First use `mapValues` and then `take`...
You should be able to figure this out now...we hope.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

<FILL IN>

Next, we create the RDD **ranks**, based on the **links** RDD. Notice that we are not interested in the values of the entries of **links**. `x[0]` takes the first element of an RDD element, which is the key. The result is a pair of the form `(key, 1.0)`. These are the seed values for the urls (articles in our case).

In [ ]:
# Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
ranks = links.map(lambda x: (x[0], 1.0))

## The iteration

Now we are ready to start the loop. The RDDs **links** and **ranks** are first joined (see the picture above), by using [`join`](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=groupbykey#pyspark.RDD.join). (The join is performed on the key - see documentation if not clear.)

The entries of this RDD look exactly as in the picture (RDD1). Notice, that each entry is a key-value pair, where the value contains two elements: the list of urls linked to, and the rank. 

Next, the rank has to be distributed over all the values. This is why the flatmap is performed on the join. Notice that *in the picture*, the values are selected (going from RDD1 to RDD2) and then the flatmap is performed. Both steps are performed together in code. The lambda function in the flatmap takes care of the fact that only the values are selected.

Elements of joinRDD have the form `(article, ((art1, art2...), 1.0)`<br>
`x[1][0]` denotes the first element of the second element: `(art1,art2...)`<br>
`x[1][1]` denotes the second element of the second element: 1.0

The helperfunction `computeContribs` divides the (initial) PageRank over the links `(art1, art2...)` and was defined above. The flatmap makes sure that we get an entry for each of the links `(art1, art2...)`. 

## Exercise 16

Now it is your turn. We created the RDD contribs (see picture). Please create **RDD 3** in the picture, called `contribsadded` in the cell below.


In [ ]:
# TODO: Replace <FILL IN> with appropriate code

for iteration in range(5): 
    joinRDD = links.join(ranks)
    contribs = joinRDD.flatMap(lambda x:computeContribs(x[1][0], x[1][1]))
   # Re-calculates URL ranks based on neighbor contributions.
    contribsAdded = <FILL IN>
    ranks = contribsAdded.mapValues(lambda rank: rank * 0.85 + 0.15)
    
# Iteration has ended
#Now take the ten first urls with ranks

pageRanks = ranks.takeOrdered(10, lambda x: -x[1])
pageRanks

## 5. Russia

Finally, we will show how multiple documents can be analyzed with the help of the RDD API.

In the data folder we have some text files on Russia. Using [NLTK](http://www.nltk.org/book/ch05.html) we will tokenize these texts and determine their Part of Speech (called PoS-Tagging).

In order read multiple files at once we use the wild character `*`. Each file that ends with `.txt` is now read into the RDD `russiaText`. We show you the first 3 entries to inspect the content.

In [ ]:
russiaText = sc.textFile("../data/russia/*.txt")
russiaText.take(5)

There are two things that are important here. First, the text is broken into lines by `sc.textFile`, we may want to replace this by a more sophisticated version from NLTK, for example. Second, we have no reference to filenames. Text from all files is lumped together in a single RDD.

We will see how to work around these two issues later. 

But first let's tokenize and determine the parts of speech. 

First we tokenize by using nltk's `word_tokenizer`. Notice, the `flatMap` instead of `map`. (If you forgot the difference between the two, feel free to try a map. But don't forget to correct this before you continue.)

In [ ]:
tokens = russiaText.flatMap(lambda x: nltk.word_tokenize(x))
tokens.take(15)

Then we determine the parts of speech. We `cache()` the result into memory.

Notice again the `flatMap`.

The output is an RDD of word-PoS pairs. The meaning of the PoS tags can be found [here](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html). 

In [ ]:
postags = tokens.flatMap(lambda x: nltk.pos_tag([x])).cache()
postags.take(15)

## Bonus exercise 1

Determine the 20 most frequent Nouns (NN) with a word length larger than one.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
# Use as many lines as you like

<FILL IN>

## Preserving filenames

How do we prevent losing file information. In our case, where files are small the answer is to read in entire files as elements of the RDD, instead of breaking them into lines.

`SparkContext` offers a method called `wholeTextFiles` that creates a Pair RDD. Each element of the RDD is a pair where the key is the file path, and the value the content of the file.

Since the path name is a bit long we trim off the path and only preserve the filename (using `ntpath.basename`).

In [ ]:
import ntpath

russiatext = sc.wholeTextFiles("../data/russia/*.txt")
rusText = russiatext.map(lambda x : (ntpath.basename(x[0]), x[1]))
rusText.take(1)


## Determining sentences

It is now also possible to let the splitting of sentences be done by an NLTK method. NLTK offers a more sophisticated version of determining sentences, instead of just splitting them on some characters.

In the code below we use `map` where we used `flatMap` before. Notice that we work with a pair RDD here and we are tokenizing only the second element. The output should also contain the file name. The result of the lambda function is a pair of the form (filename, list of tokens).

In [ ]:
sentences = rusText.map(lambda x : (x[0], nltk.sent_tokenize(x[1])))
sentences.take(1)

We can now tokenize the sentences. Because the sentences are in a list, we call `str` to convert them to a string.

In [ ]:
tokensperFile = sentences.map(lambda x : (x[0], nltk.word_tokenize(str(x[1]))))
print(tokensperFile.take(1))

Finally we determine the PoS tags per file.

In [ ]:
postagsperFile = tokensperFile.map(lambda x : (x[0], nltk.pos_tag(x[1])))
postagsperFile.take(1)

## Bonus exercise 2

Count the most 5 frequent nouns (NN) per file. As output show counts for some file. Feel free to experiment.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
# Use as many lines as you need

<FILL IN>